In [2]:
#!conda install mpl_toolkits
# todo: Bar chart with Plotly Express, MAP
#from mpl_toolkits.basemap import Basemap as Basemap

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse # shrink file size of df
# pw will calculate DISTANCES (0..N) BASED on cosine similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import StandardScaler

In [6]:
df_clean_readin = pd.read_csv('./datasets/df_clean.csv')

In [7]:
df_clean = df_clean_readin.copy()

## Colleges in datasets

In [8]:
df_clean.columns

Index(['Unnamed: 0', 'unitid', 'instnm', 'city', 'stabbr', 'numbranch', 'main',
       'highdeg', 'preddeg', 'control', 'distanceonly', 'tuitionfee_in',
       'tuitionfee_out', 'tuitionfee_prog', 'tuitfte', 'avgfacsal', 'relaffil',
       'adm_rate_all', 'satvr25', 'satvr75', 'satmt25', 'satmt75', 'satvrmid',
       'satmtmid', 'sat_avg_all', 'actcm25', 'actcm75', 'actcmmid', 'actenmid',
       'actmtmid', 'pcip01', 'pcip03', 'pcip04', 'pcip05', 'pcip09', 'pcip10',
       'pcip11', 'pcip12', 'pcip13', 'pcip14', 'pcip15', 'pcip16', 'pcip19',
       'pcip22', 'pcip23', 'pcip24', 'pcip25', 'pcip26', 'pcip27', 'pcip29',
       'pcip30', 'pcip31', 'pcip38', 'pcip39', 'pcip40', 'pcip41', 'pcip42',
       'pcip43', 'pcip44', 'pcip45', 'pcip46', 'pcip47', 'pcip48', 'pcip49',
       'pcip50', 'pcip51', 'pcip52', 'pcip54', 'ugds', 'ugds_white',
       'ugds_black', 'ugds_hisp', 'ugds_asian', 'ugds_aian', 'ugds_nhpi',
       'ugds_2mor', 'ugds_nra', 'ugds_unkn', 'ug25abv', 'pptug_ef', 'costt4_a'

## Organizing Data, Dealing with Missing Data

In [9]:
# not consider Institutions are identified by their religious affiliation 
df_clean = df_clean[df_clean['relaffil'].isnull()]
df_clean['relaffil'].fillna(value = 0, inplace=True)
df_clean.shape

(6191, 90)

In [25]:
# if the admission rate is Nan, fill it with 100%
df_clean.adm_rate_all.fillna(value = 1.0, inplace=True)
# where admission rate is 1, fill sat scores the lowest score
mask = (df_clean['adm_rate_all'] == 1.0)
df_clean['satvr25'][mask] = df_clean['satvr25'].min()
df_clean['satvr75'][mask] = df_clean['satvr75'].min()
df_clean['satmt25'][mask] = df_clean['satmt25'].min()
df_clean['satmt75'][mask] = df_clean['satmt75'].min()
df_clean['satvrmid'][mask] = df_clean['satvrmid'].min()
df_clean['satmtmid'][mask] = df_clean['satmtmid'].min()
df_clean['sat_avg_all'][mask] = df_clean['sat_avg_all'].min()
df_clean['actcm75'][mask] = df_clean['actcm75'].min()
df_clean['actcm25'][mask] = df_clean['actcm25'].min()
df_clean['actcmmid'][mask] = df_clean['actcmmid'].min()
df_clean['actmtmid'][mask] = df_clean['actmtmid'].min()
df_clean['actenmid'][mask] = df_clean['actenmid'].min()

C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyW

In [26]:
# Fill in Nan values of sat/act with mean
df_clean['satvr25'].fillna(value=df_clean['satvr25'].mean(), inplace=True)
df_clean['satvr75'].fillna(value=df_clean['satvr75'].mean(), inplace=True)
df_clean['satmt25'].fillna(value=df_clean['satmt25'].mean(), inplace=True)
df_clean['satmt75'].fillna(value=df_clean['satmt75'].mean(), inplace=True)
df_clean['satvrmid'].fillna(value=df_clean['satvrmid'].mean(), inplace=True)
df_clean['satmtmid'].fillna(value=df_clean['satmtmid'].mean(), inplace=True)
df_clean['sat_avg_all'].fillna(value=df_clean['sat_avg_all'].mean(), inplace=True)
df_clean['actcm25'].fillna(value=df_clean['actcm25'].mean(), inplace=True)
df_clean['actcm75'].fillna(value=df_clean['actcm75'].mean(), inplace=True)
df_clean['actcmmid'].fillna(value=df_clean['actcmmid'].mean(), inplace=True)
df_clean['actmtmid'].fillna(value=df_clean['actmtmid'].mean(), inplace=True)
df_clean['actenmid'].fillna(value=df_clean['actenmid'].mean(), inplace=True)

In [13]:
df_clean['ret_ft4'].fillna(value=df_clean['ret_ftl4'], inplace=True)

In [14]:
df_clean.head(2)

,Unnamed: 0,unitid,instnm,city,stabbr,numbranch,main,highdeg,preddeg,control,...,costt4_a,costt4_p,inexpfte,c150_4,ret_ft4,ret_ftl4,pctfloan,openadmp,debt_mdn,cdr3
0,0,100654,Alabama A & M University,Normal,AL,1,1,4,3,1,...,22886.0,NaN,7298.0,0.2396,0.5879,NaN,0.7697,2.0,15000,0.198
1,1,100663,University of Alabama at Birmingham,Birmingham,AL,1,1,4,3,1,...,24129.0,NaN,17235.0,0.5292,0.8436,NaN,0.5207,2.0,15000,0.059


In [15]:
df_clean.columns

Index(['Unnamed: 0', 'unitid', 'instnm', 'city', 'stabbr', 'numbranch', 'main',
       'highdeg', 'preddeg', 'control', 'distanceonly', 'tuitionfee_in',
       'tuitionfee_out', 'tuitionfee_prog', 'tuitfte', 'avgfacsal', 'relaffil',
       'adm_rate_all', 'satvr25', 'satvr75', 'satmt25', 'satmt75', 'satvrmid',
       'satmtmid', 'sat_avg_all', 'actcm25', 'actcm75', 'actcmmid', 'actenmid',
       'actmtmid', 'pcip01', 'pcip03', 'pcip04', 'pcip05', 'pcip09', 'pcip10',
       'pcip11', 'pcip12', 'pcip13', 'pcip14', 'pcip15', 'pcip16', 'pcip19',
       'pcip22', 'pcip23', 'pcip24', 'pcip25', 'pcip26', 'pcip27', 'pcip29',
       'pcip30', 'pcip31', 'pcip38', 'pcip39', 'pcip40', 'pcip41', 'pcip42',
       'pcip43', 'pcip44', 'pcip45', 'pcip46', 'pcip47', 'pcip48', 'pcip49',
       'pcip50', 'pcip51', 'pcip52', 'pcip54', 'ugds', 'ugds_white',
       'ugds_black', 'ugds_hisp', 'ugds_asian', 'ugds_aian', 'ugds_nhpi',
       'ugds_2mor', 'ugds_nra', 'ugds_unkn', 'ug25abv', 'pptug_ef', 'costt4_a'

In [27]:
df_recommender = df_clean.drop(columns=['unitid','city','stabbr','numbranch',
                                       'main','tuitionfee_in','tuitionfee_out',
                                        'tuitionfee_prog','tuitfte','relaffil',
                                        'costt4_a','costt4_p','inexpfte',
                                        'c150_4','ret_ftl4','openadmp',
                                        'debt_mdn','cdr3'
                                       ])
df_recommender.dropna(inplace=True)
df_recommender.isnull().sum().sum(), df_recommender.shape
#df_recommender.shape

(0, (2805, 72))

In [23]:
#df_recommender.columns

In [28]:
df_recommender['adm_rate_all'] = df_recommender['adm_rate_all'] * 100.

In [29]:
#df_clean[df_clean['tuitfte'].isnull()]
df_recommender_model=df_recommender.set_index('instnm')

# Standard Scaler
ss = StandardScaler()
df_recommender_model_sc = ss.fit_transform(df_recommender_model)

recommender= pairwise_distances(df_recommender_model_sc, metric='cosine')
#recommender[:5,1]

In [30]:
recommender_df = pd.DataFrame(recommender,
            index=df_recommender_model.index,
            columns=df_recommender_model.index)
recommender_df.head(2)

instnm,Alabama A & M University,University of Alabama at Birmingham,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Auburn University at Montgomery,Auburn University,Chattahoochee Valley Community College,South University-Montgomery,...,Westcliff University,Sacramento Ultrasound Institute,Bolivar Technical College,California Institute of Arts & Technology,Mechon L'hoyroa,Gwinnett Institute,Career Quest Learning Center-Mt. Pleasant,Purdue University Northwest,Ross College-Canton,Fortis College-Cutler Bay
instnm,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,0.000000,0.487657,0.603533,0.164554,0.657422,1.146818,0.418847,0.580756,1.054803,0.820719,...,1.037548,1.473667,1.432064,1.302403,1.167825,1.149788,1.509690,0.818616,1.337056,1.318801
University of Alabama at Birmingham,0.487657,0.000000,0.130734,0.448852,0.163396,1.410381,0.135634,0.114402,1.450390,1.307819,...,1.137066,1.447143,1.414506,1.341757,1.213924,1.513168,1.543981,0.349003,1.553097,1.532297


In [31]:
inputschool_name = 'Johns Hopkins University'
recommender_result = recommender_df[inputschool_name].sort_values()[1:41]
output_school_list = []
inputschool_adm_rate = df_recommender_model[df_recommender_model.index==inputschool_name]['adm_rate_all'][0]
#print(f'inputschool_adm_rate {inputschool_adm_rate} ' )
for i in range(len(recommender_result.index)):
    school_adm_rate = df_recommender_model[df_recommender_model.index==recommender_result.index[i]]['adm_rate_all'][0] 
    #print(f'{recommender_result.index[i]} : {school_adm_rate} ' )
    if np.abs(inputschool_adm_rate - school_adm_rate) < inputschool_adm_rate*0.3 :
        #print('output school list : ',output_school_list)
        output_school_list.append(recommender_result.index[i])

print('output school list : ',output_school_list)
#recommender_result

output school list :  ['Rice University', 'Washington University in St Louis', 'Vanderbilt University', 'University of Pennsylvania', 'University of Southern California', 'Cornell University', 'Swarthmore College', 'Dartmouth College']
